# Intro to Transfer Learning

## Import Packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import StandardScaler


## Prepare the Data

In [3]:
df = pd.read_csv('../Resources/meet_or_beat_US.csv')
df

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11,0.051444,0.018585
1,0.17,0.19,6,0.112955,-0.000510
2,-0.07,0.14,4,0.077167,-0.046104
3,0.48,0.51,8,-0.006130,-0.004899
4,-0.24,-0.27,9,0.089762,-0.025466
...,...,...,...,...,...
16834,2.16,0.22,1,-0.011062,-0.041509
16835,-0.16,-0.20,3,0.322034,-0.040698
16836,-0.16,-0.15,7,0.101928,0.073204
16837,-2.58,-3.45,1,0.099432,0.091058


In [4]:
X = df[['EPS','forecasted_eps','noOfEsts','before_total_returns']]
y = df[['after_total_returns']]

In [5]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Create the StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Build a Model

In [7]:
# Define the the number of inputs to the model
number_inputs = 4

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(Dropout(.2,input_shape=(hidden_nodes_layer2,)))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=24, activation="relu"))

nn.add(Dropout(.2,input_shape=(24,)))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=12, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1))

Metal device set to: Apple M1 Max


2022-03-15 10:43:03.265149: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-15 10:43:03.265322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# Summarise the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                792       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
 dense_4 (Dense)             (None, 1)                 1

In [9]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [10]:
# Fit the model
nn.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

Epoch 1/20


2022-03-15 10:43:03.425500: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-15 10:43:03.624186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


127/127 [==============================] - 1s 6ms/step - loss: 0.0149 - accuracy: 0.0079
Epoch 2/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0140 - accuracy: 0.0079
Epoch 3/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0167 - accuracy: 0.0079
Epoch 4/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0136 - accuracy: 0.0079
Epoch 5/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0138 - accuracy: 0.0079
Epoch 6/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0136 - accuracy: 0.0079
Epoch 7/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0135 - accuracy: 0.0079
Epoch 8/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0135 - accuracy: 0.0079
Epoch 9/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0138 - accuracy: 0.0079
Epoch 10/20
127/127 [==============================] - 1s 6ms/step - loss: 0.0135 - accuracy: 0.0079
Epoch 11/2

In [11]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-15 10:43:18.456149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 0s - loss: 0.0094 - accuracy: 0.0057 - 431ms/epoch - 3ms/step
Loss: 0.009384908713400364, Accuracy: 0.0057007125578820705


## Save the Model and its Weights

## Read in the Model with Weights

## Read in New Data and Work with Previously Saved Model

## Freeze Layers

## Add New Layers to Existing Model

## Fit the Revised Model